Code to extract gdp pc from the world bank api 

In [1]:
import wbgapi as wb
import pandas as pd

In [2]:
full_gdp_pc_df = wb.data.DataFrame('NY.GDP.PCAP.PP.KD', labels=True).reset_index()
full_pop_df = wb.data.DataFrame('SP.POP.TOTL', labels=False).reset_index()

In [3]:
economies_to_drop = [
    'AFE', 'AFW', 'ARB', 'CSS', 'CEB', 'EAR', 'EAS', 'EAP', 'TEA', 'EMU', 'ECS', 
    'ECA', 'TEC', 'EUU', 'FCS', 'HPC', 'HIC', 'IBD', 'IBT', 'IDB', 'IDX', 'IDA', 
    'LTE', 'LCN', 'LAC', 'TLA', 'LDC', 'LMY', 'LIC', 'LMC', 'MEA', 'MNA', 'TMN', 
    'MIC', 'NAC', 'OED', 'OSS', 'PSS', 'PST', 'PRE', 'SST', 'SAS', 'TSA', 'SSF', 
    'SSA', 'TSS', 'UMC', 'WLD'
]

# Filter both dataframes to exclude the unwanted economies
full_gdp_pc_df_clean = full_gdp_pc_df[~full_gdp_pc_df['economy'].isin(economies_to_drop)]
full_pop_df_clean = full_pop_df[~full_pop_df['economy'].isin(economies_to_drop)]

# Sort the population dataframe by 'YR2023' in descending order
full_pop_df_sorted_clean = full_pop_df_clean.sort_values(by='YR2023', ascending=False)

top_9_countries_clean = full_pop_df_sorted_clean['economy'].head(9).tolist()

# Filter the GDP dataframe for the top 9 countries
gdp_pc_top_9_clean = full_gdp_pc_df_clean[full_gdp_pc_df_clean['economy'].isin(top_9_countries_clean)]

# Drop columns with any missing values
gdp_pc_top_9_clean = gdp_pc_top_9_clean.dropna(axis=1, how='any')

# Melt the cleaned dataframe to reshape it
gdp_pc_melted_clean = gdp_pc_top_9_clean.melt(id_vars=['economy', 'Country'], 
                                               var_name='year', 
                                               value_name='gdp_pc')

gdp_pc_melted_clean['year'] = gdp_pc_melted_clean['year'].str[2:].astype(int)


In [5]:
# Convert the DataFrame into a dictionary for JSON storage
gdp_pc_melted_clean_json = gdp_pc_melted_clean.to_dict(orient='records')

# Now `gdp_pc_melted_clean_json` contains the data in a format that can be used in Vega-Lite

In [8]:
import json

# Convert DataFrame to records for JSON embedding
data_records = gdp_pc_melted_clean.to_dict(orient='records')

# Define Vega-Lite JSON specification
vega_lite_spec = {
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "data": {
        "values": data_records  # Embed data directly
    },
      "mark": "line",
  "encoding": {
    "x": {
    "field": "year",
    "type": "quantitative",
    "title": "Year"
    },
    "y": {
    "field": "gdp_pc",
    "type": "quantitative",
    "title": "GDP per Capita"
    },
    "color": {
    "field": "Country",
    "type": "nominal",
    "title": "Country"
    },
    "facet": {
    "field": "Country",
    "type": "nominal",
    "columns": 3
}
    }
}

# Save the Vega-Lite JSON specification to a file
with open('chart_6.json', 'w') as f:
    json.dump(vega_lite_spec, f, indent=2)

print("Vega-Lite JSON file created successfully: gdp_pc_melted_clean_chart.json")

Vega-Lite JSON file created successfully: gdp_pc_melted_clean_chart.json
